In [ ]:
%matplotlib inline

import datetime as dt
import itertools as it

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

In [ ]:
v2sid = 'syn10165199'
v2r = pd.read_csv(syn.get(v2sid).path, sep='\t')
v2r.head()

In [ ]:
v2 = v2r.rename(columns={
    'username':'participant_id',
    'passive_date':'dt_passive'
})

v2.head()

no columns have na values 

all columns call_hour_i-missed are 0

In [ ]:
labels = [
    'participant_id', 'dt_passive', 
    'call_duration_secs-Incoming', 'call_duration_secs-Outgoing',
    'numCalls_Incoming', 'numCalls_Missed', 'numCalls_Outgoing', 
    'numTexts-', 'numTexts-Received', 'numTexts-Sent',
    'text_length-Received', 'text_length-Sent',
    'uniqueNumbers-Incoming', 'uniqueNumbers-Missed', 'uniqueNumbers-Outgoing', 
    'uniqueNumbersTexted-', 'uniqueNumbersTexted-Received', 'uniqueNumbersTexted-Sent'
]
v2 = v2.reindex(labels, axis=1).fillna(0)

v2 = v2.rename(columns={
    'call_duration_secs-Incoming':'callDuration_incoming', 
    'call_duration_secs-Outgoing':'callDuration_outgoing',
    'numCalls_Incoming':'callCount_incoming',
    'numCalls_Missed':'callCount_missed',
    'numCalls_Outgoing':'callCount_outgoing', 
    'numTexts-':'textCount', 
    'numTexts-Received':'textCount_received', 
    'numTexts-Sent':'textCount_sent',
    'text_length-Received':'textLength_received', 
    'text_length-Sent':'textLength_sent',
    'uniqueNumbers-Incoming':'uniqueNumbers_calls_incoming', 
    'uniqueNumbers-Missed':'uniqueNumbers_calls_missed', 
    'uniqueNumbers-Outgoing':'uniqueNumbers_calls_outgoing', 
    'uniqueNumbersTexted-':'uniqueNumbers_texts', 
    'uniqueNumbersTexted-Received':'uniqueNumbers_texts_received',
    'uniqueNumbersTexted-Sent':'uniqueNumbers_texts_sent'
})

v2.head()

In [12]:
cheat = []
for c in v2.columns:
    cheat.append(
        (c, str(v2[c].dtype).replace('object', 'str').replace('float64', 'float').replace('int64', 'int'))
    )
pd.DataFrame(cheat)

,0,1
0,participant_id,str
1,dt_passive,str
2,callDuration_incoming,int
3,callDuration_outgoing,int
4,callCount_incoming,float
5,callCount_missed,float
6,callCount_outgoing,float
7,textCount,int
8,textCount_received,int
9,textCount_sent,int


In [9]:
final = syn.store(Table(
    Schema(
            name='Passive Features (v2)',
            columns=as_table_columns(v2), 
            parent='syn10848316'),
        v2
    )
)

In [10]:
final = syn.setProvenance(
    'syn17060502',
    activity=Activity(
        name='Generate V2 Table Data',
        description='Process the data collected during study ',
        used=[v2sid],
        executed=[
            dict(
                name='IPython Notebook',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_V2PassiveFeatures_datafiles.ipynb'
            )
        ]
    )
)